# **Support Vector Machine Regression (SVR)**
## **Regresja metodą wektorów nośnych**

---

### **Overview / Przegląd**

Support Vector Machine Regression (SVR) extends SVM to regression problems. Instead of finding a hyperplane that separates classes, SVR finds a hyperplane that best fits the data within a specified tolerance (epsilon).

Regresja metodą wektorów nośnych (SVR) rozszerza SVM na problemy regresji. Zamiast znajdować hiperpłaszczyznę, która rozdziela klasy, SVR znajduje hiperpłaszczyznę, która najlepiej dopasowuje się do danych w ramach określonej tolerancji (epsilon).

### **Mathematical Foundation / Podstawy matematyczne**

The SVR optimization problem:
Problem optymalizacyjny SVR:

$$\min_{w,b,\xi,\xi^*} \frac{1}{2}||w||^2 + C\sum_{i=1}^{n}(\xi_i + \xi_i^*)$$

Subject to:
Przy ograniczeniach:
- $y_i - w^T\phi(x_i) - b \leq \varepsilon + \xi_i$
- $w^T\phi(x_i) + b - y_i \leq \varepsilon + \xi_i^*$
- $\xi_i, \xi_i^* \geq 0$

Where:
Gdzie:
- $\varepsilon$ = epsilon (tolerance for errors)
- $C$ = regularization parameter
- $\xi_i, \xi_i^*$ = slack variables


In [ ]:
# Import required libraries / Importowanie wymaganych bibliotek
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_regression, load_diabetes, load_boston
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

# Set style for plots / Ustawienie stylu wykresów
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully / Biblioteki zaimportowane pomyślnie")


## **1. Dataset Creation and Exploration**
### *Tworzenie i eksploracja zbioru danych*


CHANGE DATASET!!!

In [ ]:
# Create synthetic regression dataset / Tworzenie syntetycznego zbioru danych regresji
np.random.seed(42)
X_synthetic, y_synthetic = make_regression(
    n_samples=200, 
    n_features=1, 
    noise=10, 
    random_state=42
)

# Add some outliers / Dodanie kilku wartości odstających
outlier_indices = np.random.choice(len(X_synthetic), 10, replace=False)
y_synthetic[outlier_indices] += np.random.normal(0, 50, 10)

print(f"Synthetic dataset shape / Kształt syntetycznego zbioru: {X_synthetic.shape}")
print(f"Target range / Zakres zmiennej docelowej: [{y_synthetic.min():.2f}, {y_synthetic.max():.2f}]")

# Visualize synthetic data / Wizualizacja syntetycznych danych
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(X_synthetic, y_synthetic, alpha=0.7, color='blue')
plt.xlabel('Feature')
plt.ylabel('Target')
plt.title('Synthetic Regression Dataset\nSyntetyczny zbiór danych regresji')
plt.grid(True, alpha=0.3)

# Load diabetes dataset / Wczytanie zbioru danych diabetes
diabetes = load_diabetes()
X_diabetes = diabetes.data
y_diabetes = diabetes.target

print(f"\nDiabetes dataset shape / Kształt zbioru diabetes: {X_diabetes.shape}")
print(f"Feature names / Nazwy cech: {diabetes.feature_names}")

# Visualize diabetes data (first feature) / Wizualizacja danych diabetes (pierwsza cecha)
plt.subplot(1, 2, 2)
plt.scatter(X_diabetes[:, 0], y_diabetes, alpha=0.7, color='red')
plt.xlabel('Age (standardized)')
plt.ylabel('Diabetes Progression')
plt.title('Diabetes Dataset\nZbiór danych Diabetes')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Split synthetic data / Podział syntetycznych danych
X_train, X_test, y_train, y_test = train_test_split(
    X_synthetic, y_synthetic, test_size=0.3, random_state=42
)

# Compare different SVR kernels / Porównanie różnych jąder SVR
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

svr_results = []

for i, kernel in enumerate(kernels):
    # Create and train SVR / Tworzenie i trenowanie SVR
    svr = SVR(kernel=kernel, C=1.0, gamma='scale')
    svr.fit(X_train, y_train)
    
    # Make predictions / Przewidywania
    y_pred = svr.predict(X_test)
    
    # Calculate metrics / Obliczenie metryk
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    
    svr_results.append({
        'Kernel': kernel,
        'MSE': mse,
        'R²': r2,
        'MAE': mae
    })
    
    # Plot results / Rysowanie wyników
    X_plot = np.linspace(X_synthetic.min(), X_synthetic.max(), 300).reshape(-1, 1)
    y_plot = svr.predict(X_plot)
    
    axes[i].scatter(X_train, y_train, alpha=0.6, color='blue', label='Training data')
    axes[i].scatter(X_test, y_test, alpha=0.6, color='red', label='Test data')
    axes[i].plot(X_plot, y_plot, color='green', linewidth=2, label='SVR prediction')
    axes[i].set_title(f'SVR - {kernel.capitalize()} Kernel\\nR² = {r2:.3f}, MSE = {mse:.2f}')
    axes[i].set_xlabel('Feature')
    axes[i].set_ylabel('Target')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Display results table / Wyświetlenie tabeli wyników
results_df = pd.DataFrame(svr_results)
print("\\nSVR Kernel Comparison / Porównanie jąder SVR:")
print("=" * 50)
print(results_df.to_string(index=False, float_format='%.4f'))
